In [1]:
import pandas as pd

In [2]:
dataset_path = '/content/Child 1 Genome.csv'
dataset = pd.read_csv(dataset_path)

<ipython-input-2-55e3a89287d3>:2: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(dataset_path)


In [3]:

print("Dataset Information:")
dataset.info()
print("\nFirst 5 rows of the dataset:")
print(dataset.head())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601802 entries, 0 to 601801
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   # rsid      601802 non-null  object
 1   chromosome  601802 non-null  object
 2   position    601802 non-null  object
 3   genotype    601802 non-null  object
dtypes: object(4)
memory usage: 18.4+ MB

First 5 rows of the dataset:
        # rsid chromosome position genotype
0   rs12564807          1   734462       AA
1    rs3131972          1   752721       AG
2  rs148828841          1   760998       AC
3   rs12124819          1   776546       AG
4  rs115093905          1   787173       GG


In [4]:
dataset['position'] = pd.to_numeric(dataset['position'], errors='coerce')
dataset = dataset.dropna(subset=['position'])
dataset = dataset.sort_values(by=['chromosome', 'position'])

In [5]:
sequence_data = dataset.groupby('chromosome')['genotype'].apply(list).reset_index()
print("\nProcessed Sequence Data:")
print(sequence_data.head())


Processed Sequence Data:
  chromosome                                           genotype
0          3  [AA, AG, CC, GG, TT, GG, CC, AA, GT, GT, AA, G...
1          4  [GG, CC, TT, TT, TT, CC, AA, AG, CC, TT, TT, G...
2          5  [GG, CC, GG, GG, GG, AA, GG, CT, CC, GG, TT, C...
3          6  [AA, GG, CT, CC, CC, CT, GG, GT, AA, GG, GT, G...
4          7  [CT, CT, AC, TT, CT, CT, GG, CT, CC, CC, GG, A...


In [6]:
from collections import defaultdict
from itertools import combinations


In [7]:
frequent_patterns = []
one_itemsets = defaultdict(int)

In [8]:
sequences = sequence_data['genotype'].tolist()

In [9]:
print(sequence_data.head())

  chromosome                                           genotype
0          3  [AA, AG, CC, GG, TT, GG, CC, AA, GT, GT, AA, G...
1          4  [GG, CC, TT, TT, TT, CC, AA, AG, CC, TT, TT, G...
2          5  [GG, CC, GG, GG, GG, AA, GG, CT, CC, GG, TT, C...
3          6  [AA, GG, CT, CC, CC, CT, GG, GT, AA, GG, GT, G...
4          7  [CT, CT, AC, TT, CT, CT, GG, CT, CC, CC, GG, A...


In [10]:
for seq in sequences:
        unique_items = set(seq)
        for item in unique_items:
            one_itemsets[item] += 1

In [11]:
min_support = 5

In [12]:
print("Min Support:", min_support)

Min Support: 5


In [13]:
current_patterns = {k: v for k, v in one_itemsets.items() if v >= min_support}
frequent_patterns.extend(current_patterns.keys())

In [14]:
min_item_support = 2  # Adjust as needed
item_counts = defaultdict(int)
for seq in sequences:
    for item in seq:
        item_counts[item] += 1
frequent_items = {item for item, count in item_counts.items() if count >= min_item_support}
sequences = [[item for item in seq if item in frequent_items] for seq in sequences]


In [15]:
max_pattern_length = 5
k = 2
while current_patterns and k <= max_pattern_length:
    # Existing logic
    k += 1


In [16]:
k = 2
while current_patterns:
    candidate_patterns = defaultdict(int)

    # Generate candidate k-sequences
    items = list(current_patterns.keys())
    candidates = [tuple(sorted(set(a) | set(b))) for a, b in combinations(items, 2) if len(set(a) | set(b)) == k]
    candidates = set(candidates)  # Remove duplicates

    # Debug statement for candidates
    print(f"Iteration {k}: {len(current_patterns)} patterns, {len(candidates)} candidates")

    # Count support for each candidate pattern
    for seq in sequences:
        seq_set = set(seq)
        for candidate in candidates:
            if set(candidate).issubset(seq_set):
                candidate_patterns[candidate] += 1

    # Filter by minimum support
    current_patterns = {k: v for k, v in candidate_patterns.items() if v >= min_support}
    frequent_patterns.extend(current_patterns.keys())

    k += 1



Iteration 2: 14 patterns, 21 candidates


In [17]:
if 'genotype' in sequence_data.columns:
    sequences = sequence_data['genotype'].tolist()
else:
    print("Error: 'genotype' column not found in the data. Please check the dataset.")

# Set a minimum support threshold (e.g., 5)
min_support = 5

In [18]:
def gsp_algorithm(sequences, min_support):
    """
    Implementation of the GSP algorithm for mining sequential patterns.
    :param sequences: List of sequences (e.g., list of genotype sequences for each chromosome).
    :param min_support: Minimum support threshold (e.g., frequency).
    :return: List of frequent sequential patterns.
    """
    # Initialize variables
    frequent_patterns = []
    one_itemsets = defaultdict(int)

In [19]:
from itertools import combinations

def gsp_algorithm_optimized(sequences, min_support):
    frequent_patterns = []
    one_itemsets = defaultdict(int)

    # Count occurrences of individual items (1-sequences)
    for seq in sequences:
        unique_items = set(seq)
        for item in unique_items:
            one_itemsets[item] += 1

    # Filter by minimum support
    current_patterns = {k: v for k, v in one_itemsets.items() if v >= min_support}
    frequent_patterns.extend(current_patterns.keys())

    # Iteratively find k-sequences
    k = 2
    while current_patterns:
        candidate_patterns = defaultdict(int)

        # Generate candidate k-sequences by combining previous patterns
        items = list(current_patterns.keys())

        # Efficient combination of patterns: Instead of generating all pairs upfront, we generate them on-the-fly
        for a, b in combinations(items, 2):
            combined_pattern = tuple(sorted(set(a) | set(b)))

            # Only generate and count patterns of length k
            if len(combined_pattern) == k:
                # Check the support for the combined pattern
                for seq in sequences:
                    seq_set = set(seq)
                    if set(combined_pattern).issubset(seq_set):
                        candidate_patterns[combined_pattern] += 1

        # Filter by minimum support
        current_patterns = {k: v for k, v in candidate_patterns.items() if v >= min_support}
        frequent_patterns.extend(current_patterns.keys())

        k += 1

    return frequent_patterns

# Example of how to apply this optimization
min_support = 5
gsp_patterns = gsp_algorithm_optimized(sequences, min_support)

print("\nFrequent Patterns Discovered by GSP (Optimized):")
print(gsp_patterns[:10])  # Show the first 10 patterns for brevity



Frequent Patterns Discovered by GSP (Optimized):
['GG', 'AG', 'CT', 'GT', 'CC', 'AA', 'AC', 'TT', '--', 'DD']


In [20]:
import concurrent.futures
from collections import defaultdict
from itertools import combinations

def extract_kmers(sequence, k):
    """Extract all k-mers from a DNA sequence."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

def count_support_for_pattern(pattern, sequences, min_support, k):
    """Count the support of a single k-mer pattern in sequences."""
    candidate_count = 0
    for seq in sequences:
        kmers_in_seq = extract_kmers(seq, k)  # Extract k-mers for this sequence
        if pattern in kmers_in_seq:
            candidate_count += 1
    return (pattern, candidate_count)  # Return both the pattern and its count

def spade_algorithm_parallel(sequences, min_support, k):
    """Parallel SPADE algorithm for DNA sequence mining."""
    frequent_patterns = []
    one_itemsets = defaultdict(list)

    # Step 1: Identify single k-mers (1-sequences) and their occurrences
    for seq_id, seq in enumerate(sequences):
        kmers_in_seq = extract_kmers(seq, k)  # Extract k-mers from the sequence
        for kmer in kmers_in_seq:
            one_itemsets[kmer].append((seq_id, kmer))

    # Step 2: Filter by minimum support
    current_patterns = {kmer: v for kmer, v in one_itemsets.items() if len(v) >= min_support}
    frequent_patterns.extend(current_patterns.keys())

    # Step 3: Iteratively mine longer k-sequences (k-mers of length > k)
    while current_patterns:
        candidate_patterns = defaultdict(int)  # Store pattern counts

        # Generate candidate k-sequences by combining previous patterns
        items = list(current_patterns.keys())

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []
            for i in range(len(items)):
                for j in range(i + 1, len(items)):
                    # Combine two k-mers into a (k+1)-mer pattern
                    combined_pattern = items[i] + items[j][-1]  # Merge last nucleotide of second k-mer
                    futures.append(executor.submit(count_support_for_pattern, combined_pattern, sequences, min_support, k + 1))

            for future in concurrent.futures.as_completed(futures):
                result = future.result()  # (pattern, count)
                if result:
                    pattern, count = result
                    if count >= min_support:
                        candidate_patterns[pattern] = count  # Store pattern and its support count

        # Update current patterns with those that meet the minimum support
        current_patterns = candidate_patterns
        frequent_patterns.extend(current_patterns.keys())  # Add the frequent patterns

        k += 1  # Increment the length of patterns to find

    return frequent_patterns

# Example usage:
min_support = 3  # Adjust based on your dataset
sequences = [
    'ATCGATCG',  # Example DNA sequences
    'GCTAGCTA',
    'ATCGGCTA',
    'CGATCGAT'
]

# Run the SPADE algorithm in parallel for 2-mers (subsequences of length 2)
k = 2
spade_patterns = spade_algorithm_parallel(sequences, min_support, k)

# Display results
print("Frequent Patterns Discovered by SPADE (Parallelized):")
print(spade_patterns[:10])  # Show the first 10 patterns for brevity


Frequent Patterns Discovered by SPADE (Parallelized):
['AT', 'TC', 'CG', 'GA', 'GC', 'CT', 'TA', 'TCG', 'ATC']


In [21]:
import pandas as pd
import concurrent.futures
from collections import defaultdict
from itertools import combinations

# Step 2: Function to extract k-mers (subsequences) from the genotype list
def extract_kmers(sequence, k):
    """Extract all k-mers from a sequence of genotypes."""
    return [tuple(sequence[i:i+k]) for i in range(len(sequence) - k + 1)]

def count_support_for_pattern(pattern, sequences, min_support, k):
    """Count the support of a single k-mer pattern in sequences."""
    candidate_count = 0
    for seq in sequences:
        kmers_in_seq = extract_kmers(seq, k)  # Extract k-mers for this sequence
        if pattern in kmers_in_seq:
            candidate_count += 1
    return (pattern, candidate_count)  # Return both the pattern and its count

def spade_algorithm_parallel(sequences, min_support, k):
    """Parallel SPADE algorithm for mining genotype sequence patterns."""
    frequent_patterns = []
    one_itemsets = defaultdict(list)

    # Step 1: Identify single k-mers (1-sequences) and their occurrences
    for seq_id, seq in enumerate(sequences):
        kmers_in_seq = extract_kmers(seq, k)  # Extract k-mers from the sequence
        for kmer in kmers_in_seq:
            one_itemsets[kmer].append((seq_id, kmer))

    # Step 2: Filter by minimum support
    current_patterns = {kmer: v for kmer, v in one_itemsets.items() if len(v) >= min_support}
    frequent_patterns.extend(current_patterns.keys())

    # Step 3: Iteratively mine longer k-sequences (k-mers of length > k)
    while current_patterns:
        candidate_patterns = defaultdict(int)  # Store pattern counts

        # Generate candidate k-sequences by combining previous patterns
        items = list(current_patterns.keys())

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []
            for i in range(len(items)):
                for j in range(i + 1, len(items)):
                    # Combine two k-mers into a (k+1)-mer pattern by merging the tuples
                    combined_pattern = items[i] + items[j][1:]  # Merge all elements of the second k-mer (excluding its first element)
                    futures.append(executor.submit(count_support_for_pattern, combined_pattern, sequences, min_support, k + 1))

            for future in concurrent.futures.as_completed(futures):
                result = future.result()  # (pattern, count)
                if result:
                    pattern, count = result
                    if count >= min_support:
                        candidate_patterns[pattern] = count  # Store pattern and its support count

        # Update current patterns with those that meet the minimum support
        current_patterns = candidate_patterns
        frequent_patterns.extend(current_patterns.keys())  # Add the frequent patterns

        k += 1  # Increment the length of patterns to find

    return frequent_patterns

# Example usage
min_support = 3  # Adjust based on your dataset
k = 2  # Start with 2-mers (subsequences of length 2)

# Run the SPADE algorithm in parallel
spade_patterns = spade_algorithm_parallel(sequences, min_support, k)

# Display results
print("Frequent Patterns Discovered by SPADE (Parallelized):")
print(spade_patterns[:10])  # Show the first 10 patterns for brevity



Frequent Patterns Discovered by SPADE (Parallelized):
[('A', 'T'), ('T', 'C'), ('C', 'G'), ('G', 'A'), ('G', 'C'), ('C', 'T'), ('T', 'A'), ('T', 'C', 'G'), ('A', 'T', 'C')]


In [22]:
import pandas as pd



# Display the first few rows of the dataset to understand the structure
print(dataset.head())

# Since the 'genotype' column contains individual genotypes as strings (e.g., 'AA', 'AG', etc.),
# we can directly extract them into sequences
sequences = dataset.groupby('chromosome')['genotype'].apply(list).tolist()

# Verify the sequences after conversion
print(sequences[:5])  # Print the first 5 sequences to check



           # rsid chromosome     position genotype
131072  rs9325377          3  196506153.0       AA
131073  rs4916546          3  196510533.0       AG
131074  rs7646247          3  196519209.0       CC
131075  rs6583176          3  196525578.0       GG
131076  rs4916555          3  196538648.0       TT
[['AA', 'AG', 'CC', 'GG', 'TT', 'GG', 'CC', 'AA', 'GT', 'GT', 'AA', 'GG', 'CC', 'AG', 'GG', 'AG', 'CC', 'CC', 'CT', 'GG', 'AG', 'TT', 'AA', 'CT', 'AA', 'AG', 'TT', 'CT', '--', 'CT', 'AG', 'AC', 'GG', 'AC', 'TT', 'GG', 'GG', 'GG', 'AA', 'AG', 'CC', 'AG', 'CC', 'GT', 'CC', 'AG', 'GG', 'TT', 'CC', 'CC', 'TT', 'CC', 'TT', 'GG', 'CC', 'CC', 'AG', 'CC', 'TT', 'AA', 'GG', 'AA', 'AA', 'CC', 'TT', 'TT', 'GG', 'TT', 'AA', 'AA', 'TT', 'GG', 'AA', 'CC', 'TT', 'TT', 'CC', 'AA', 'AA', 'AA', 'TT', 'AA', 'CT', 'GT', 'CC', 'AG', 'AG', 'GG', 'CC', 'CT', 'AG', 'GG', 'TT', 'CC', 'GG', 'AG', 'AA', 'AA', '--', 'GG', 'GG', 'GG', 'AA', 'AG', 'CT', 'CT', 'AG', 'GG', 'AG', 'AG', 'CT', 'CT', 'AC', 'AG', 'CC', 'A

In [23]:
from collections import defaultdict

def prefixspan(sequences, min_support, max_pattern_length=float('inf')):
    """
    PrefixSpan algorithm for mining sequential patterns.
    :param sequences: List of sequences, each being a list of items (e.g., genotype list).
    :param min_support: Minimum support threshold.
    :param max_pattern_length: Maximum length of patterns to mine (default is unlimited).
    :return: List of frequent sequential patterns.
    """
    # Step 1: Function to find frequent 1-sequences
    def find_frequent_1seq(sequences, min_support):
        item_support = defaultdict(int)
        for seq in sequences:
            for item in seq:
                item_support[item] += 1
        frequent_1seq = {item: count for item, count in item_support.items() if count >= min_support}
        return frequent_1seq

    # Step 2: Recursively mine patterns with prefixes
    def mine_with_prefix(prefix, sequences, min_support, max_pattern_length):
        frequent_patterns = []

        # Step 2.1: Find frequent 1-sequences in the sequences
        frequent_1seq = find_frequent_1seq(sequences, min_support)

        # Step 2.2: Add frequent 1-sequences to the result and mine further with prefix
        for item, count in frequent_1seq.items():
            new_prefix = prefix + [item]
            frequent_patterns.append((new_prefix, count))

            if len(new_prefix) < max_pattern_length:
                # Filter subsequences that contain the new prefix
                projected_sequences = []
                for seq in sequences:
                    if item in seq:
                        projected_sequences.append(seq[seq.index(item)+1:])

                # Recurse with the new prefix and the filtered subsequences
                frequent_patterns.extend(mine_with_prefix(new_prefix, projected_sequences, min_support, max_pattern_length))

        return frequent_patterns

    # Step 3: Start mining from the empty prefix
    return mine_with_prefix([], sequences, min_support, max_pattern_length)

# Example usage:
sequences = [['AA', 'AG', 'CC', 'GG'], ['GG', 'CC', 'TT'], ['AA', 'AG', 'CC', 'GG', 'TT']]
min_support = 2
max_pattern_length = 3  # Set maximum pattern length to 3 for example
patterns = prefixspan(sequences, min_support, max_pattern_length)

print("Frequent Patterns Discovered by PrefixSpan:")
for pattern, count in patterns:
    print(f"Pattern: {pattern}, Support: {count}")


Frequent Patterns Discovered by PrefixSpan:
Pattern: ['AA'], Support: 2
Pattern: ['AA', 'AG'], Support: 2
Pattern: ['AA', 'AG', 'CC'], Support: 2
Pattern: ['AA', 'AG', 'GG'], Support: 2
Pattern: ['AA', 'CC'], Support: 2
Pattern: ['AA', 'CC', 'GG'], Support: 2
Pattern: ['AA', 'GG'], Support: 2
Pattern: ['AG'], Support: 2
Pattern: ['AG', 'CC'], Support: 2
Pattern: ['AG', 'CC', 'GG'], Support: 2
Pattern: ['AG', 'GG'], Support: 2
Pattern: ['CC'], Support: 3
Pattern: ['CC', 'GG'], Support: 2
Pattern: ['CC', 'TT'], Support: 2
Pattern: ['GG'], Support: 3
Pattern: ['GG', 'TT'], Support: 2
Pattern: ['TT'], Support: 2


In [24]:
import pandas as pd
from collections import defaultdict

# PrefixSpan algorithm for mining sequential patterns
def prefixspan(sequences, min_support, max_pattern_length=float('inf')):
    """
    PrefixSpan algorithm for mining sequential patterns.
    :param sequences: List of sequences, each being a list of items (e.g., genotype list).
    :param min_support: Minimum support threshold.
    :param max_pattern_length: Maximum length of patterns to mine (default is unlimited).
    :return: List of frequent sequential patterns.
    """
    # Step 1: Function to find frequent 1-sequences
    def find_frequent_1seq(sequences, min_support):
        item_support = defaultdict(int)
        for seq in sequences:
            for item in seq:
                item_support[item] += 1
        frequent_1seq = {item: count for item, count in item_support.items() if count >= min_support}
        return frequent_1seq

    # Step 2: Recursively mine patterns with prefixes
    def mine_with_prefix(prefix, sequences, min_support, max_pattern_length):
        frequent_patterns = []

        # Step 2.1: Find frequent 1-sequences in the sequences
        frequent_1seq = find_frequent_1seq(sequences, min_support)

        # Step 2.2: Add frequent 1-sequences to the result and mine further with prefix
        for item, count in frequent_1seq.items():
            new_prefix = prefix + [item]
            frequent_patterns.append((new_prefix, count))

            if len(new_prefix) < max_pattern_length:
                # Filter subsequences that contain the new prefix
                projected_sequences = []
                for seq in sequences:
                    if item in seq:
                        projected_sequences.append(seq[seq.index(item)+1:])

                # Recurse with the new prefix and the filtered subsequences
                frequent_patterns.extend(mine_with_prefix(new_prefix, projected_sequences, min_support, max_pattern_length))

        return frequent_patterns

    # Step 3: Start mining from the empty prefix
    return mine_with_prefix([], sequences, min_support, max_pattern_length)

# Load your dataset from a CSV file
# Replace 'your_dataset.csv' with the actual path to your file
dataset = pd.read_csv('/content/Child 1 Genome.csv')

# Verify the dataset structure (check the first few rows)
print(dataset.head())

# Step 1: Extract the 'genotype' column (assuming each genotype is a single string like 'AA', 'AG', etc.)
# The dataset contains individual items like 'AA', 'AG', 'GG', etc. You don't need eval for this case.
sequences = dataset['genotype'].tolist()  # Each row is a single genotype, no need for eval

# Convert the genotype sequences into a list of lists, where each sequence is treated as a list of items
# For example, ['AA', 'AG', 'CC'] becomes [['AA', 'AG', 'CC']]
sequences = [[genotype] for genotype in sequences]

# Set minimum support threshold
min_support = 2
max_pattern_length = 3  # Set the maximum pattern length for mining

# Run PrefixSpan
patterns = prefixspan(sequences, min_support, max_pattern_length)

# Display results
print("Frequent Patterns Discovered by PrefixSpan:")
for pattern, count in patterns:
    print(f"Pattern: {pattern}, Support: {count}")


<ipython-input-24-30e79464973d>:51: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('/content/Child 1 Genome.csv')


        # rsid chromosome position genotype
0   rs12564807          1   734462       AA
1    rs3131972          1   752721       AG
2  rs148828841          1   760998       AC
3   rs12124819          1   776546       AG
4  rs115093905          1   787173       GG
Frequent Patterns Discovered by PrefixSpan:
Pattern: ['AA'], Support: 88533
Pattern: ['AG'], Support: 65852
Pattern: ['AC'], Support: 15060
Pattern: ['GG'], Support: 111969
Pattern: ['CT'], Support: 66450
Pattern: ['CC'], Support: 112019
Pattern: ['GT'], Support: 15125
Pattern: ['CG'], Support: 790
Pattern: ['genotype'], Support: 2
Pattern: ['--'], Support: 9223
Pattern: ['TT'], Support: 88489
Pattern: ['AT'], Support: 422
Pattern: ['II'], Support: 2381
Pattern: ['DD'], Support: 840
Pattern: ['DI'], Support: 33
Pattern: ['G'], Support: 6026
Pattern: ['A'], Support: 5874
Pattern: ['T'], Support: 5548
Pattern: ['C'], Support: 6446
Pattern: ['I'], Support: 542
Pattern: ['D'], Support: 178


In [25]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv('/content/Child 1 Genome.csv')

# Check the first few rows of the dataset
print(dataset.head())

# Step 1: Remove rows with invalid or missing genotypes
# For example, we remove rows where the genotype column is empty or contains invalid values like '--' or 'genotype'
valid_genotypes = ['AA', 'AG', 'AC', 'GG', 'CT', 'CC', 'GT', 'CG', 'TT', 'AT', 'II', 'DD', 'DI', 'G', 'A', 'T', 'C', 'I', 'D']
dataset = dataset[dataset['genotype'].isin(valid_genotypes)]

# Step 2: Convert the 'genotype' column into a list of lists (as required by PrefixSpan)
sequences = dataset['genotype'].tolist()
sequences = [[genotype] for genotype in sequences]

# Check the cleaned sequences
print(sequences[:10])

# Now you can proceed with running the PrefixSpan algorithm as before
min_support = 2
max_pattern_length = 3  # Set maximum pattern length for mining

patterns = prefixspan(sequences, min_support, max_pattern_length)

# Display results
print("Frequent Patterns Discovered by PrefixSpan:")
for pattern, count in patterns:
    print(f"Pattern: {pattern}, Support: {count}")


<ipython-input-25-58bc43dc6438>:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('/content/Child 1 Genome.csv')


        # rsid chromosome position genotype
0   rs12564807          1   734462       AA
1    rs3131972          1   752721       AG
2  rs148828841          1   760998       AC
3   rs12124819          1   776546       AG
4  rs115093905          1   787173       GG
[['AA'], ['AG'], ['AC'], ['AG'], ['GG'], ['GG'], ['AA'], ['AC'], ['CT'], ['AG']]
Frequent Patterns Discovered by PrefixSpan:
Pattern: ['AA'], Support: 88533
Pattern: ['AG'], Support: 65852
Pattern: ['AC'], Support: 15060
Pattern: ['GG'], Support: 111969
Pattern: ['CT'], Support: 66450
Pattern: ['CC'], Support: 112019
Pattern: ['GT'], Support: 15125
Pattern: ['CG'], Support: 790
Pattern: ['TT'], Support: 88489
Pattern: ['AT'], Support: 422
Pattern: ['II'], Support: 2381
Pattern: ['DD'], Support: 840
Pattern: ['DI'], Support: 33
Pattern: ['G'], Support: 6026
Pattern: ['A'], Support: 5874
Pattern: ['T'], Support: 5548
Pattern: ['C'], Support: 6446
Pattern: ['I'], Support: 542
Pattern: ['D'], Support: 178


In [26]:
def freespam(sequences, min_support, max_pattern_length=float('inf')):
    """
    FreeSpan algorithm to mine sequential patterns.
    :param sequences: List of sequences (e.g., genotype data).
    :param min_support: Minimum support threshold.
    :param max_pattern_length: Maximum length of patterns to mine (default is infinite).
    :return: List of frequent sequential patterns with their support counts.
    """
    frequent_patterns = []

    # Step 1: Find frequent 1-sequences
    frequent_1seq = find_frequent_1seq(sequences, min_support)
    # Add the 1-sequences as tuples (pattern, count)
    for item, count in frequent_1seq.items():
        frequent_patterns.append(([item], count))  # Ensure it's a tuple (pattern, count)

    # Step 2: Recursively mine longer sequences
    def mine_with_prefix(prefix, sequences, min_support, max_pattern_length):
        nonlocal frequent_patterns

        # Extract subsequences that contain the prefix
        subsequences = []
        for seq in sequences:
            subsequences.extend(extract_subsequences(seq, prefix))

        # Step 2.1: Find frequent 1-sequences in the subsequences
        frequent_1seq_in_subsequences = find_frequent_1seq(subsequences, min_support)

        # Step 2.2: Add frequent 1-sequences to the result and recursively extend them
        for item, count in frequent_1seq_in_subsequences.items():
            new_prefix = prefix + [item]
            frequent_patterns.append((new_prefix, count))  # Add the frequent pattern as a tuple (pattern, count)

            if len(new_prefix) < max_pattern_length:
                # Recurse with the new prefix
                mine_with_prefix(new_prefix, subsequences, min_support, max_pattern_length)

    # Step 3: Mine patterns with different prefixes
    for item, count in frequent_1seq.items():
        prefix = [item]
        mine_with_prefix(prefix, sequences, min_support, max_pattern_length)

    return frequent_patterns


In [27]:
# Debugging: Print the structure of patterns before unpacking
print("Frequent Patterns List:", patterns)


Frequent Patterns List: [(['AA'], 88533), (['AG'], 65852), (['AC'], 15060), (['GG'], 111969), (['CT'], 66450), (['CC'], 112019), (['GT'], 15125), (['CG'], 790), (['TT'], 88489), (['AT'], 422), (['II'], 2381), (['DD'], 840), (['DI'], 33), (['G'], 6026), (['A'], 5874), (['T'], 5548), (['C'], 6446), (['I'], 542), (['D'], 178)]


In [28]:
def freespam(sequences, min_support, max_pattern_length=float('inf')):
    """
    FreeSpan algorithm to mine sequential patterns.
    :param sequences: List of sequences (e.g., genotype data).
    :param min_support: Minimum support threshold.
    :param max_pattern_length: Maximum length of patterns to mine (default is infinite).
    :return: List of frequent sequential patterns with their support counts.
    """
    frequent_patterns = []

    # Step 1: Find frequent 1-sequences
    frequent_1seq = find_frequent_1seq(sequences, min_support)

    # Add frequent 1-sequences as (pattern, count) tuples
    for item, count in frequent_1seq.items():
        frequent_patterns.append(([item], count))  # Wrap the item in a list to represent it as a sequence

    # Debugging: Print the patterns after the 1-sequences step
    print("Frequent 1-sequences (after Step 1):")
    for pattern in frequent_patterns:
        print(pattern)

    # Step 2: Recursively mine longer sequences
    def mine_with_prefix(prefix, sequences, min_support, max_pattern_length):
        nonlocal frequent_patterns

        # Extract subsequences that contain the prefix
        subsequences = []
        for seq in sequences:
            subsequences.extend(extract_subsequences(seq, prefix))

        # Step 2.1: Find frequent 1-sequences in the subsequences
        frequent_1seq_in_subsequences = find_frequent_1seq(subsequences, min_support)

        # Step 2.2: Add frequent 1-sequences to the result and recursively extend them
        for item, count in frequent_1seq_in_subsequences.items():
            new_prefix = prefix + [item]  # Extend the prefix with the new item
            frequent_patterns.append((new_prefix, count))  # Add the frequent pattern with its count

            # Debugging: Print the newly discovered patterns
            print(f"New pattern: {new_prefix}, Support: {count}")

            if len(new_prefix) < max_pattern_length:
                # Recurse with the new prefix
                mine_with_prefix(new_prefix, subsequences, min_support, max_pattern_length)

    # Step 3: Mine patterns with different prefixes
    for item, count in frequent_1seq.items():
        prefix = [item]  # Start with the frequent 1-sequence as the initial prefix
        mine_with_prefix(prefix, sequences, min_support, max_pattern_length)

    # Final check to ensure all items are in the form of (list, count) tuples
    frequent_patterns = [(pattern, count) for pattern, count in frequent_patterns if isinstance(pattern, list)]

    return frequent_patterns


In [29]:
# Step 4: Display results
print("Frequent Patterns Discovered by FreeSpan:")
for pattern, count in patterns:
    print(f"Pattern: {pattern}, Support: {count}")


Frequent Patterns Discovered by FreeSpan:
Pattern: ['AA'], Support: 88533
Pattern: ['AG'], Support: 65852
Pattern: ['AC'], Support: 15060
Pattern: ['GG'], Support: 111969
Pattern: ['CT'], Support: 66450
Pattern: ['CC'], Support: 112019
Pattern: ['GT'], Support: 15125
Pattern: ['CG'], Support: 790
Pattern: ['TT'], Support: 88489
Pattern: ['AT'], Support: 422
Pattern: ['II'], Support: 2381
Pattern: ['DD'], Support: 840
Pattern: ['DI'], Support: 33
Pattern: ['G'], Support: 6026
Pattern: ['A'], Support: 5874
Pattern: ['T'], Support: 5548
Pattern: ['C'], Support: 6446
Pattern: ['I'], Support: 542
Pattern: ['D'], Support: 178
